In [25]:
import os

import ir_datasets
import whoosh.index as index
from whoosh.filedb.filestore import FileStorage

In [2]:
dataset = ir_datasets.load("beir/msmarco/test")

In [5]:
for q in dataset.qrels_iter():
    print(q)
    break

TrecQrel(query_id='19335', doc_id='1017759', relevance=0, iteration='0')


In [70]:
dataset.docs_count()

8841823

In [6]:
docs = list(dataset.docs_iter())
queries = list(dataset.queries_iter())
qrels = list(dataset.qrels_iter())

[INFO] [starting] building docstore
[INFO] [starting] opening zip file                                              
[INFO] [finished] opening zip file s]                                        
docs_iter: 100%|█████████████████| 8841823/8841823 [00:54<00:00, 161000.17doc/s]
[INFO] [finished] docs_iter: [00:54] [8841823doc] [160999.50doc/s]
[INFO] [finished] building docstore [54.92s]


In [16]:
from whoosh.fields import SchemaClass, TEXT, KEYWORD, ID, STORED

class MySchema(SchemaClass):
    name = ID(stored=True)
    content = TEXT

In [20]:
if not os.path.exists("indexdir"):
    os.mkdir("indexdir")

ix = index.create_in("indexdir", MySchema())

In [28]:
ix = index.open_dir("indexdir")
writer = ix.writer()

In [31]:
docs[0]

GenericDoc(doc_id='0', text='The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.')

In [33]:
i = 0
while i < 4:
    doc = docs[i]
    writer.add_document(name=f"doc_{doc.doc_id}", content=doc.text)
    i += 1
writer.commit()

In [34]:
searcher = ix.searcher()

In [59]:
docs[2]

GenericDoc(doc_id='2', text='Essay on The Manhattan Project - The Manhattan Project The Manhattan Project was to see if making an atomic bomb possible. The success of this project would forever change the world forever making it known that something this powerful can be manmade.')

In [57]:
from whoosh.qparser import QueryParser

qp = QueryParser("content", schema=ix.schema)
q = qp.parse("Manhattan Project")
results = searcher.search(q, limit=2)

In [58]:
results[0]

<Hit {'name': 'doc_2'}>

In [64]:
qrels[0]

TrecQrel(query_id='19335', doc_id='1017759', relevance=0, iteration='0')